# Проект по анализу данных с сайта РИА Новости

### Часть 1.1
### Реализация парсера

In [1]:
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from dataclasses import dataclass
from scipy.stats import mannwhitneyu
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime
import random
from wordcloud import WordCloud, STOPWORDS
import requests
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

sns.set(style="darkgrid")
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kukof\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
SLEEP = random.randint(1,5)
BASE_URL = 'https://lenta.ru/rubrics/'
TOPICS = ['world/crime','world/society','world/politic', 'world/accident', 'world/conflict','russia/society','russia/politic', 'russia/accident', 'russia/regions', 'russia/moscow','forces/investigations','forces/violation', 'forces/police', 'forces/crimerussia','media/internet', 'media/hackers', 'media/soc_network', 'media/memes','media/press', 'media/tv', 'media/factchecking','sport/football', 'sport/boxing', 'sport/winter','sport/other','sport/hockey','sport/auto','sport/olympiad','sport/fitness','culture/kino', 'culture/series', 'culture/music/', 'culture/books/', 'culture/art/','culture/theatre','culture/photographic','science/science', 'science/inrussia','science/cosmos', 'science/mil','science/history', 'science/natural' , 'science/future', 'science/digital','science/gadget', 'science/games' , 'science/soft','economics/economy', 'economics/companies', 'economics/markets', 'economics/finance', 'economics/business_climate', 'economics/social','economics/crypto','economics/investments','realty/city','/realty/home', 'realty/region','realty/climate','realty/transport','life/people','life/animals','life/food','life/accident','life/persons','life/lucky','life/anomalies','travel/rus','travel/world','travel/events','travel/accident','travel/opinion','style/look','style/exterior','style/phenomenon','style/luxury','style/personality','wellness/dietology','wellness/selfcare','wellness/interior_ideas','wellness/mental_health','wellness/relationships']
stts = ['/10', '/12', '/14']

In [13]:
@dataclass
class Article:
    id: str = None
    url: str = None
    source: str = None
    title: str = None
    content: str = None
    datetime: str = None

In [14]:
# set webdriver params
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('no-sandbox')
chrome_options.add_argument('disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

# driver = webdriver.Chrome('./chromedriver')

In [15]:
def get_pages():

    """Load and scroll pages"""

    items, topics_order = [], []

    for topic in tqdm(TOPICS):
        try:
            old_size = len(items)
            for st_i in stts:
                URL = BASE_URL + topic + st_i
                print(URL)
                driver.get(URL)
                time.sleep(SLEEP)

            # push to list 20 next articles
                driver.execute_script("document.getElementsByClassName('loadmore__button')[0].click()")
                time.sleep(SLEEP)

            # scroll page to automatically load more articles

                try:
                    #driver.execute_script(f'window.scrollTo(0, document.body.scrollHeight - 1200)')
                        # Scroll down to bottom
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(SLEEP)
                except:
                    pass

            # find all pages

            #req = requests.get(URL)
            #soup = BeautifulSoup(req.content, 'html.parser')
            #soup.find_all('li', {'class' : 'rubric-page__item _news'})
                html = driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                items += soup.find_all('li', {'class' : 'rubric-page__item _news'})

            # number of pages can not be multiple of deepth*20
            # that's why we count topics_order dynamically
                new_size = len(items)
                if new_size > old_size:
                    topics_order.extend([topic] * (new_size - old_size))
        except:
            pass

    return items, topics_order

In [16]:
def parse_page(page):
    """Extract from page desired fields"""

    # Create article data class object
    article = Article()

    # article id
    article.id = page.find('a', {'class': 'card-full-news _subrubric'})['href']

    # article url
    article.url = 'https://lenta.ru/' + article.id

    # load page
    driver.get(article.url)
    time.sleep(random.randint(1,5))
    html = driver.page_source

    #soup = BeautifulSoup(html, "html.parser")
    response = requests.get(article.url)

    soup = BeautifulSoup(response.content, 'html.parser')

    # article source
    article.source = article.url[8: article.url.find('.')]

    # process article title
    title = soup.find('span', {'class': 'topic-body__title'})
    title_2 = soup.find('div', {'class': 'topic-body__title-yandex'})

    if title:
        article.title = title.text
    else:
        article.title = title_2.text if title_2 else ''


    # article content
    article.content = soup.find('div', {'class': 'topic-body__content'}).text

    # article datetime
    article.datetime = soup.find('a', {'class': 'topic-header__item topic-header__time'})['href']

    return article

In [17]:
# get pages and topics
pages, topics_order = get_pages()
# with open('pages.txt', 'w') as f:
#     for page in pages:
#         f.write("%s\n" % page)лъ

  0%|          | 0/82 [00:00<?, ?it/s]

https://lenta.ru/rubrics/world/crime/10
https://lenta.ru/rubrics/world/crime/12
https://lenta.ru/rubrics/world/crime/14


  1%|          | 1/82 [01:06<1:29:42, 66.45s/it]

https://lenta.ru/rubrics/world/society/10
https://lenta.ru/rubrics/world/society/12
https://lenta.ru/rubrics/world/society/14


  2%|▏         | 2/82 [02:14<1:30:01, 67.52s/it]

...

 98%|█████████▊| 80/82 [1:17:35<01:49, 54.72s/it]

https://lenta.ru/rubrics/wellness/mental_health/10
https://lenta.ru/rubrics/wellness/mental_health/12
https://lenta.ru/rubrics/wellness/mental_health/14


 99%|█████████▉| 81/82 [1:18:30<00:54, 54.76s/it]

https://lenta.ru/rubrics/wellness/relationships/10
https://lenta.ru/rubrics/wellness/relationships/12
https://lenta.ru/rubrics/wellness/relationships/14


100%|██████████| 82/82 [1:19:24<00:00, 58.11s/it]


In [18]:
topics_order

['world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/

In [19]:
pages

[<li class="rubric-page__item _news"><a class="card-full-news _subrubric" href="/news/2023/10/26/v-ssha-utochnili-chislo-zhertv-lyuistonskogo-strelka/"><h3 class="card-full-news__title">В США уточнили число жертв льюистонского стрелка</h3><div class="card-full-news__info"><time class="card-full-news__info-item card-full-news__date">18:29, 26 октября 2023</time><span class="card-full-news__info-item card-full-news__rubric">Мир</span></div></a></li>,
 <li class="rubric-page__item _news"><a class="card-full-news _subrubric" href="/news/2023/10/26/on-slyshal-golosa-v-golove-chto-izvestno-o-lichnosti-voennogo-ustroivshego-samoe-massovoe-ubiystvo-v-ssha-v-2023-godu/"><h3 class="card-full-news__title">«Он слышал голоса в голове» Что известно о стрелке, устроившем самое массовое убийство в США в 2023 году?</h3><div class="card-full-news__info"><time class="card-full-news__info-item card-full-news__date">14:06, 26 октября 2023</time><span class="card-full-news__info-item card-full-news__rubric"

In [20]:
# parse each page and get desired attributes
   # few pages are differ significantly from others
# using try/except we can ignore them
data, topics_order_fixed = [], []
for num, page in enumerate(tqdm(pages)):
    try:
        res = parse_page(page)
        data.append(res)
        topics_order_fixed.append(topics_order[num])
    except:
        pass

driver.close()

100%|██████████| 13728/13728 [23:33:06<00:00,  6.18s/it]  


In [25]:
df_10_12_14 = pd.DataFrame(data=data)
df_10_12_14['topic'] = topics_order_fixed
df_10_12_14

,id,url,source,title,content,datetime,topic
0,/news/2023/10/26/v-ssha-utochnili-chislo-zhert...,https://lenta.ru//news/2023/10/26/v-ssha-utoch...,lenta,В США уточнили число жертв льюистонского стрелка,18 человек погибли в результате массового расс...,/2023/10/26/,world/crime
1,/news/2023/10/26/on-slyshal-golosa-v-golove-ch...,https://lenta.ru//news/2023/10/26/on-slyshal-g...,lenta,«Он слышал голоса в голове» Что известно о стр...,40-летнего американца Роберт Карда подозревают...,/2023/10/26/,world/crime
2,/news/2023/10/26/politsiya-ustanovila-lichnost...,https://lenta.ru//news/2023/10/26/politsiya-us...,lenta,Полиция установила личность подозреваемого в с...,Полиция установила личность подозреваемого в с...,/2023/10/26/,world/crime
3,/news/2023/10/26/politsiya-predupredila-ob-opa...,https://lenta.ru//news/2023/10/26/politsiya-pr...,lenta,Полиция предупредила об опасности льюистонског...,"Полиция американского города Льюистон, штат Мэ...",/2023/10/26/,world/crime
4,/news/2023/10/26/lyuistonskogo-strelka-zaderzh...,https://lenta.ru//news/2023/10/26/lyuistonskog...,lenta,Льюистонского стрелка задержали,"Полиция штата Мэн задержала мужчину, предполож...",/2023/10/26/,world/crime
...,...,...,...,...,...,...,...
13720,/news/2023/10/13/sex_testosteron/,https://lenta.ru//news/2023/10/13/sex_testoste...,lenta,Врач назвал первые симптомы лишающей секса нех...,Повышенная утомляемость и раздражительность — ...,/2023/10/13/,science/future
13721,/news/2023/10/12/nazvana-prichina-tyagi-muzhch...,https://lenta.ru//news/2023/10/12/nazvana-pric...,lenta,Названа причина тяги мужчин к секс-технологиям,Мужчины намного чаще женщин пользуются техноло...,/2023/10/12/,science/future
13722,/news/2023/10/12/teenager/,https://lenta.ru//news/2023/10/12/teenager/,lenta,Родителям пьющих подростков дали совет,"Родителям, которые столкнулись с тем, что их р...",/2023/10/12/,science/future
13723,/news/2023/10/12/marriage/,https://lenta.ru//news/2023/10/12/marriage/,lenta,Названа неочевидная причина измен в счастливом...,"Семейный психотерапевт Ричард Хоган рассказал,...",/2023/10/12/,science/future


In [28]:
df_10_12_14[df_10_12_14.duplicated()]

,id,url,source,title,content,datetime,topic
56,/news/2023/10/26/v-ssha-utochnili-chislo-zhert...,https://lenta.ru//news/2023/10/26/v-ssha-utoch...,lenta,В США уточнили число жертв льюистонского стрелка,18 человек погибли в результате массового расс...,/2023/10/26/,world/crime
57,/news/2023/10/26/on-slyshal-golosa-v-golove-ch...,https://lenta.ru//news/2023/10/26/on-slyshal-g...,lenta,«Он слышал голоса в голове» Что известно о стр...,40-летнего американца Роберт Карда подозревают...,/2023/10/26/,world/crime
58,/news/2023/10/26/politsiya-ustanovila-lichnost...,https://lenta.ru//news/2023/10/26/politsiya-us...,lenta,Полиция установила личность подозреваемого в с...,Полиция установила личность подозреваемого в с...,/2023/10/26/,world/crime
59,/news/2023/10/26/politsiya-predupredila-ob-opa...,https://lenta.ru//news/2023/10/26/politsiya-pr...,lenta,Полиция предупредила об опасности льюистонског...,"Полиция американского города Льюистон, штат Мэ...",/2023/10/26/,world/crime
60,/news/2023/10/26/lyuistonskogo-strelka-zaderzh...,https://lenta.ru//news/2023/10/26/lyuistonskog...,lenta,Льюистонского стрелка задержали,"Полиция штата Мэн задержала мужчину, предполож...",/2023/10/26/,world/crime
...,...,...,...,...,...,...,...
13720,/news/2023/10/13/sex_testosteron/,https://lenta.ru//news/2023/10/13/sex_testoste...,lenta,Врач назвал первые симптомы лишающей секса нех...,Повышенная утомляемость и раздражительность — ...,/2023/10/13/,science/future
13721,/news/2023/10/12/nazvana-prichina-tyagi-muzhch...,https://lenta.ru//news/2023/10/12/nazvana-pric...,lenta,Названа причина тяги мужчин к секс-технологиям,Мужчины намного чаще женщин пользуются техноло...,/2023/10/12/,science/future
13722,/news/2023/10/12/teenager/,https://lenta.ru//news/2023/10/12/teenager/,lenta,Родителям пьющих подростков дали совет,"Родителям, которые столкнулись с тем, что их р...",/2023/10/12/,science/future
13723,/news/2023/10/12/marriage/,https://lenta.ru//news/2023/10/12/marriage/,lenta,Названа неочевидная причина измен в счастливом...,"Семейный психотерапевт Ричард Хоган рассказал,...",/2023/10/12/,science/future


In [29]:
topics_order

['world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/crime',
 'world/

In [30]:
df_10_12_14.to_pickle('df_10_12_14.p', compression='gzip')